In [64]:
from math import radians, cos, sin, asin, sqrt
import re
import pandas as pd
import numpy as np
import statistics
from heapq import nlargest
import folium
from scipy.spatial.distance import cdist
from typing import List, Tuple
from gmplot import GoogleMapPlotter
from gmplot import gmplot
from lat_lon_parser import parse
import os

In [65]:
#Load data#

def coordinate_weights_loader(file_name, country, coordinate_reference = 'coordinates', weight_reference='weights_gdp'):
    coords_list = []
    weights_list = []
    coords_list = import_from_excel_col(file_name, sheet_name=country, reference_col=coordinate_reference)
    weights_list = import_from_excel_col(file_name, sheet_name=country, reference_col=weight_reference)
    latitudes, longitudes = parse_coordinates(coords_list=coords_list)
    latitudes_degrees, longitudes_degrees = coords_converter(latitudes=latitudes, longitudes=longitudes)
    coordinates = list(zip(latitudes_degrees, longitudes_degrees))
    return coordinates, weights_list

def coordinate_loader(file_name, country, coordinate_reference = 'coordinates', weight_reference='weights_gdp'):
    coords_list = []
    coords_list = import_from_excel_col(file_name, sheet_name=country, reference_col=coordinate_reference)
    latitudes, longitudes = parse_coordinates(coords_list=coords_list)
    latitudes_degrees, longitudes_degrees = coords_converter(latitudes=latitudes, longitudes=longitudes)
    coordinates = list(zip(latitudes_degrees, longitudes_degrees))
    return coordinates

##helper functions##
def import_from_excel_col(file_path, sheet_name, reference_col='weights_gdp'):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    values = df[reference_col].tolist()
    return values

def parse_coordinates(coords_list):
    north_south = []
    east_west = []
    for coord in coords_list:
        lat, lon = coord.split()
        if "N" in lat:
            north_south.append(lat)
        elif "S" in lat:
            north_south.append(lat)
        else:
            raise ValueError(f"Invalid latitude format: {lat}")
        
        if "E" in lon:
            east_west.append(lon)
        elif "W" in lon:
            east_west.append(lon)
        else:
            raise ValueError(f"Invalid longitude format: {lon}")
        
    return north_south, east_west

def coords_converter(latitudes, longitudes):
    output_lats = []
    output_lons = []
    for lat in latitudes:
        tmp = parse(lat)
        output_lats.append(tmp)

    for lon in longitudes:
        tmp = parse(lon)
        output_lons.append(tmp)
    
    if len(output_lats) == len(output_lons):
        return output_lats, output_lons
    else: 
        print("coordinate lengths of latitudes in input list did not match length of longitudes")

#Geometric Median and Distance Computations
def weighted_geometric_median_haversine2(coords, weights, eps=1e-5, max_iter=1000):
    """
    Calculate the weighted geometric median point of a list of GPS coordinates using haversine distance.
    
    Parameters:
    coords (list): A list of tuples containing the latitude and longitude values of each GPS coordinate.
    weights (list): A list of weights for each GPS coordinate.
    eps (float): The convergence threshold for the iterative algorithm.
    max_iter (int): The maximum number of iterations to perform.
    
    Returns:
    tuple: The weighted geometric median point as a tuple containing the latitude and longitude values.
    """
    num_coords = len(coords)
    coords_rad = np.radians(coords)
    # Initialize the weighted geometric median estimate as the weighted arithmetic mean of the coordinates
    wgm = tuple([np.average(coords_rad[:,i], weights=weights) for i in range(2)])
    # Iterate until convergence or maximum number of iterations
    for i in range(max_iter):
        num_x, num_y, den = 0, 0, 0
        for i, coord in enumerate(coords):
            dist = haversine_distance(coord, wgm)
            num_x += weights[i]*coord[0]/dist
            num_y += weights[i]*coord[1]/dist
            den += weights[i]/dist
        new_wgm = tuple([num_x/den, num_y/den])
        if haversine_distance(wgm, new_wgm) < eps:
            break
        wgm = new_wgm
    return wgm

def haversine_distance(coords1, coords2):
    """
    Calculate the great circle distance between two points on the earth (specified in decimal degrees)
    using the Haversine formula.
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [coords1[0], coords1[1], coords2[0], coords2[1]])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r


def distance_matrix(coords_list):
    """
    Create a distance matrix between all coordinates in the given list using great circle distance.
    """
    num_coords = len(coords_list)
    dist_mat = np.zeros((num_coords, num_coords))
    for i in range(num_coords):
        for j in range(i+1, num_coords):
            dist = haversine_distance(coords_list[i], coords_list[j])
            dist_mat[i, j] = dist
            dist_mat[j, i] = dist
    return dist_mat

def weighted_distance_matrix(coords_list, weights, distance_fn=haversine_distance):
    """
    Calculate a weighted distance matrix for a list of GPS coordinates using a custom distance function.

    Parameters:
    coords_list (list): A list of tuples containing the latitude and longitude values of each GPS coordinate.
    weights (list): A list of weights for each coordinate.
    distance_fn (function): A function that takes in two coordinates as tuples and returns their distance.

    Returns:
    ndarray: A 2D numpy array representing the weighted distance matrix.
    """
    num_coords = len(coords_list)
    distances = np.zeros((num_coords, num_coords))
    for i in range(num_coords):
        for j in range(i, num_coords):
            dist = distance_fn(coords_list[i], coords_list[j])
            distances[i][j] = dist
            distances[j][i] = dist
    weighted_distances = np.multiply(distances, np.outer(weights, weights))
    return weighted_distances




In [66]:
#import data
filename ='data.xlsx'
country = 'colombia'
coords, weights = coordinate_weights_loader(filename, country)
print(coords)
print(weights)
wgm = weighted_geometric_median_haversine2(coords=coords, weights=weights)
print(wgm)

[(4.609722222222222, -74.08166666666666), (6.683333333333334, -75.56666666666666), (3.9333333333333336, -76.51666666666667), (6.666666666666667, -73.45), (5.0, -74.16666666666667), (10.65, -74.96666666666667), (8.883333333333333, -74.31666666666666), (3.6222222222222222, -73.15), (5.75, -73.1), (4.05, -75.25), (9.233333333333333, -73.51666666666667), (2.283333333333333, -76.85), (8.366666666666667, -75.7), (5.283333333333333, -75.35), (2.8, -75.45), (5.016666666666667, -75.91666666666667), (8.016666666666667, -72.88333333333334), (1.6, -77.86666666666666), (5.433333333333334, -71.5), (10.183333333333334, -74.23333333333333), (11.55, -72.35), (4.433333333333334, -75.68333333333334), (9.033333333333333, -75.15), (6.616666666666667, -70.98333333333333), (6.316666666666666, -77.0), (1.0333333333333334, -73.9), (0.5666666666666667, -75.65), (4.609722222222222, -74.08166666666666), (1.9833333333333334, -71.93333333333334), (-1.6072222222222223, -71.36749999999999), (4.633333333333333, -69.23

In [67]:
#set parameters for computation

filename='data.xlsx'
countries = import_from_excel_col(filename, sheet_name='country_list', reference_col='countries')
print(countries)
wgms_list = []
my_eps =1e-10

#compute values

for country in countries:
    coords, weights = coordinate_weights_loader(filename, country)
    print("The coordinate and weights lists for "+country+" are: ")
    print(coords)
    print(weights)
    wgm = weighted_geometric_median_haversine2(coords=coords, weights=weights,eps=my_eps)
    concatenated = f"{wgm[0]:.6f}, {wgm[1]:.6f}"
    print("The weighted geometric median for " + country + " is: " + concatenated)
    wgms_list.append(wgm)

print(wgms_list)

['germany', 'colombia', 'china', 'russia', 'poland', 'indonesia', 'south_africa', 'india', 'usa', 'kazakhstan', 'australia']
The coordinate and weights lists for germany are: 
[(51.46666666666667, 7.55), (49.078611111111115, 11.385555555555555), (48.53777777777778, 9.04111111111111), (52.75611111111111, 9.393055555555556), (50.60805555555556, 9.028333333333334), (52.519999999999996, 13.405000000000001), (49.91305555555555, 7.45), (51.026944444444446, 13.358888888888888), (53.55, 10.0), (54.47, 9.51388888888889), (52.36194444444445, 13.008055555555556), (51.96666666666667, 11.466666666666667), (50.861111111111114, 11.066666666666668), (53.61666666666667, 12.7), (49.38333333333333, 6.833333333333333), (53.34722222222222, 8.59138888888889)]
[0.20535866069610664, 0.18527285513400854, 0.15012472585377767, 0.08844655188396673, 0.08472732130539393, 0.04563619147428004, 0.04543174581747596, 0.037671212735228034, 0.035486724895403644, 0.029269336428211147, 0.022029719676998266, 0.01879499729879

In [68]:
# Create dataframe for wgms
wgm_df = pd.DataFrame({
    "country": countries,
    "wgm": wgms_list
})

# Print dataframe
print(wgm_df)

         country                                       wgm
0        germany    (50.64072490149433, 9.102413536450362)
1       colombia   (5.004178779025576, -74.28357854314487)
2          china   (32.00986623920785, 115.72989638324015)
3         russia    (55.46136984433171, 43.53478976670728)
4         poland   (51.64950457877499, 19.428714044082746)
5      indonesia  (-6.143453623468695, 107.85706057633907)
6   south_africa  (-26.01666666667249, 28.127222222222013)
7          india   (20.073472071422188, 77.99320318908593)
8            usa    (38.48995963078638, -88.6490647444514)
9     kazakhstan    (47.56861682291284, 70.73160930344051)
10     australia  (-29.914678608728437, 135.7612523059649)


In [69]:
#import regions of interest and create df

countries = import_from_excel_col(file_path=filename, sheet_name='coal_regions', reference_col='country')
regions = import_from_excel_col(file_path=filename, sheet_name='coal_regions', reference_col='region')
converted_coords = coordinate_loader(file_name=filename, country='coal_regions')


regions_df = pd.DataFrame({"country": countries, "region": regions, "converted_coordinates": converted_coords })
print(regions_df)


         country                     region  \
0       colombia                      Cesar   
1       colombia                 La Guajira   
2        germany                        NRW   
3   south_africa                 Mpumalanga   
4         poland                   łódzkie    
5          china                   Shandong   
6          china                     Shanxi   
7          china                    Shaanxi   
8          china             Inner Mongolia   
9          china                   Xinjiang   
10     australia                 Queensland   
11     australia            New South Wales   
12     indonesia            East Kalimantan   
13     indonesia           South Kalimantan   
14    kazakhstan            Karagandinskaya   
15    kazakhstan                   Pavlodar   
16         india               Chhattisgarh   
17         india                     Odisha   
18         india             Madhya Pradesh   
19         india                  Jharkhand   
20        pol

In [70]:
#Merge dfs

merged_df = pd.merge(wgm_df, regions_df, on="country")

print(merged_df)

         country                                       wgm  \
0        germany    (50.64072490149433, 9.102413536450362)   
1        germany    (50.64072490149433, 9.102413536450362)   
2       colombia   (5.004178779025576, -74.28357854314487)   
3       colombia   (5.004178779025576, -74.28357854314487)   
4          china   (32.00986623920785, 115.72989638324015)   
5          china   (32.00986623920785, 115.72989638324015)   
6          china   (32.00986623920785, 115.72989638324015)   
7          china   (32.00986623920785, 115.72989638324015)   
8          china   (32.00986623920785, 115.72989638324015)   
9         russia    (55.46136984433171, 43.53478976670728)   
10        russia    (55.46136984433171, 43.53478976670728)   
11        russia    (55.46136984433171, 43.53478976670728)   
12        russia    (55.46136984433171, 43.53478976670728)   
13        russia    (55.46136984433171, 43.53478976670728)   
14        russia    (55.46136984433171, 43.53478976670728)   
15      

In [71]:
#compute unweighted distance from region to wgm

wgms_list = merged_df['wgm'].tolist()
coords_list = merged_df['converted_coordinates'].tolist()
distances_list = []

for i in range(len(wgms_list)):
    tmp_dist = haversine_distance(wgms_list[i], coords_list[i])
    distances_list.append(tmp_dist)

print(distances_list)

merged_df['distance_to_wgm'] = distances_list
print(merged_df)

[142.1515328725739, 301.94706491173315, 477.80958063051935, 758.2872307138363, 578.0040269138353, 698.4816839747865, 790.7323980042888, 1526.904486812129, 2925.2615635945226, 5115.103754746403, 4000.311801919356, 5827.701056732213, 5724.541737628321, 6370.104783813332, 5573.141129720252, 5586.7896303587195, 6080.003872604915, 4587.942491990577, 6507.453187184312, 4393.608307031947, 3771.1611882212337, 2954.3826143622578, 2927.7854636596926, 3283.422376197303, 5.421342493851867, 230.56496076012186, 1177.7824789778674, 910.0138091434562, 203.47357617892962, 460.8598633255686, 672.8467459559879, 435.50947688484024, 884.4266377697595, 1665.5339610060973, 986.8311116223641, 148.7647480166907, 736.0369512675722, 262.5815565109388, 179.22823410412823, 660.3173155820484, 1198.2214112246552, 1095.9065466847696]
         country                                       wgm  \
0        germany    (50.64072490149433, 9.102413536450362)   
1        germany    (50.64072490149433, 9.102413536450362)   


In [72]:
#save output data

merged_df.to_excel('output.xlsx', sheet_name='output', index=False)